In [1]:
import zipfile
import os
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import boto3
import pandas as pd
from io import BytesIO
from io import StringIO
import json
import requests 

In [2]:
# Initialize S3 client
s3 = boto3.client('s3')

# Function to load data 
bucket_name = 'ml-stack-759578648427-us-west-2'

def from_s3(bucket_name, file_name):
    response = s3.get_object(Bucket=bucket_name, Key=file_name)
    ext = os.path.splitext(file_name)[1]
    if ext == '.gz':
        data = pd.read_csv(BytesIO(response['Body'].read()), compression='gzip')
    elif ext == '.csv':
        data = pd.read_csv(BytesIO(response['Body'].read()))
    else:
        data = pd.read_excel(BytesIO(response['Body'].read()), sheet_name='DX_to_CCSR_Mapping', header=1)
    return data

# Import Modeling Set

In [3]:
file_name = 'MIMIC/Cleaned/df_omr_pt_ad.csv'
df = from_s3(bucket_name, file_name) 
df['charttime'] = pd.to_datetime(df['charttime'])
subjects = df['subject_id'].unique()
df.head()

,subject_id,charttime,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,gender,anchor_age,insurance,language,marital_status,race
0,10000826,2146-12-08 22:22:00,63.786,166.608,116.368,73.125,F,32,Private,English,SINGLE,WHITE
1,10000898,2187-09-26 08:00:00,62.251,162.011,126.000,76.000,F,79,NaN,NaN,NaN,NaN
2,10000980,2185-10-24 00:00:00,63.242,173.000,150.000,90.000,F,72,NaN,NaN,NaN,NaN
3,10001122,2142-04-08 10:48:00,62.926,170.767,131.007,72.597,F,70,NaN,NaN,NaN,NaN
4,10001401,2133-12-15 11:04:00,65.450,167.941,146.537,60.630,F,91,Medicare,English,MARRIED,WHITE


In [4]:
df[df.subject_id  == 10000980]

,subject_id,charttime,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,gender,anchor_age,insurance,language,marital_status,race
2,10000980,2185-10-24,63.242,173.000,150.000,90.000,F,72,NaN,NaN,NaN,NaN


# + Diagnosis

Use ICD 9 to 10 crosswalk from: https://github.com/JeanNikiema/mimicinicd11/tree/main

Use broad category map from: https://hcup-us.ahrq.gov/toolssoftware/ccsr/dxccsr.jsp

In [11]:
file_name = 'MIMIC/Cleaned/icd_code_map.csv'
icd_code_map = from_s3(bucket_name, file_name) 

file_name = 'MIMIC/admissions.csv.gz'
ad = from_s3(bucket_name, file_name) 

file_name = 'MIMIC/diagnoses_icd.csv.gz'
diag = from_s3(bucket_name, file_name) 

# filter to possible patients only
diag = diag[diag['subject_id'].isin(subjects)]

# join addmission to get discharge time - prevent data leakage
diag = diag.merge(ad[['subject_id', 'hadm_id', 'dischtime']], on=['subject_id', 'hadm_id'], how='left')
del ad

# merge icd_code_mapping
diag_mapped = diag.merge(icd_code_map, on='icd_code', how='left')

# % matched
matched = diag_mapped[~diag_mapped.ccsr_category.isna()]
num_matched = diag_mapped[~diag_mapped.ccsr_category.isna()].icd_code.nunique() / diag_mapped.icd_code.nunique()
print(f"Matched {num_matched:.2%} codes")

# keep broad codes only
diag_mapped = diag_mapped[['subject_id', 'dischtime', 'ccsr_category']].drop_duplicates()
# fill diag_mapped ccsr_category with 99
diag_mapped.ccsr_category.fillna(99, inplace=True)
diag_mapped.head()

Matched 98.49% codes


/tmp/ipykernel_4293/3049866623.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  diag_mapped.ccsr_category.fillna(99, inplace=True)


,subject_id,dischtime,ccsr_category
0,10000826,2146-12-12 16:30:00,DIG019
1,10000826,2146-12-12 16:30:00,MBD017
2,10000826,2146-12-12 16:30:00,RSP002
3,10000826,2146-12-12 16:30:00,SYM006
5,10000826,2146-12-12 16:30:00,GEN004


In [6]:
diag_pivot = diag_mapped.groupby(['subject_id', 'dischtime', 'ccsr_category']).size().reset_index()
diag_pivot = diag_pivot.pivot(index=['subject_id', 'dischtime'], columns='ccsr_category', values=0).reset_index()
diag_pivot.fillna(0, inplace=True)
diag_pivot.head()

ccsr_category,subject_id,dischtime,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,...,SYM009,SYM010,SYM011,SYM012,SYM013,SYM014,SYM015,SYM016,SYM017,SYM018
0,10000826,2146-12-12 16:30:00,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,10000826,2146-12-24 19:55:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,10000826,2147-01-02 17:45:00,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,10000980,2188-01-05 17:30:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,10000980,2189-07-03 03:00:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [7]:
# merge to sub/charttime data set
sub_time = df[['subject_id', 'charttime']].drop_duplicates()
df_diag = sub_time.merge(diag_pivot, on=['subject_id'], how='left')

# filter disch less than chartime
df_diag['dischtime'] = pd.to_datetime(df_diag['dischtime'])
df_diag['charttime'] = pd.to_datetime(df_diag['charttime'])
df_diag = df_diag[(df_diag.dischtime <= df_diag.charttime) | (df_diag.dischtime.isna() == True)]

# drop disch
df_diag.drop(columns=['dischtime'], inplace=True)

# count all diagnoses in medical hisotry up to charttime
df_diag = df_diag.groupby(['subject_id', 'charttime']).sum().reset_index()
df_diag.fillna(0, inplace=True)
df_diag.head()

,subject_id,charttime,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,...,SYM009,SYM010,SYM011,SYM012,SYM013,SYM014,SYM015,SYM016,SYM017,SYM018
0,10000898,2187-09-26 08:00:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,10001122,2142-04-08 10:48:00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,10001401,2133-12-15 11:04:00,0.000,1.000,0.000,1.000,2.000,0.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000
3,10001401,2134-10-09 13:45:00,0.000,1.000,0.000,1.000,2.000,0.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000
4,10001401,2135-09-20 13:04:00,0.000,1.000,0.000,1.000,2.000,0.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000


In [8]:
# join to final data set 
df_omr_pt_ad_diag = df.merge(df_diag, on=['subject_id', 'charttime'], how='left')
df_omr_pt_ad_diag.head()

,subject_id,charttime,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,gender,anchor_age,insurance,language,...,SYM009,SYM010,SYM011,SYM012,SYM013,SYM014,SYM015,SYM016,SYM017,SYM018
0,10000826,2146-12-08 22:22:00,63.786,166.608,116.368,73.125,F,32,Private,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000898,2187-09-26 08:00:00,62.251,162.011,126.000,76.000,F,79,NaN,NaN,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,10000980,2185-10-24 00:00:00,63.242,173.000,150.000,90.000,F,72,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001122,2142-04-08 10:48:00,62.926,170.767,131.007,72.597,F,70,NaN,NaN,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,10001401,2133-12-15 11:04:00,65.450,167.941,146.537,60.630,F,91,Medicare,English,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000


In [9]:
df_omr_pt_ad_diag.describe()

,subject_id,charttime,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,anchor_age,99,BLD001,BLD002,...,SYM009,SYM010,SYM011,SYM012,SYM013,SYM014,SYM015,SYM016,SYM017,SYM018
count,44918.000,44918,44916.000,44916.000,44915.000,44915.000,44918.000,30209.000,30209.000,30209.000,...,30209.000,30209.000,30209.000,30209.000,30209.000,30209.000,30209.000,30209.000,30209.000,30209.000
mean,15008362.972,2155-11-13 19:40:29.742196736,64.810,175.400,128.159,72.680,64.246,1.192,0.339,0.030,...,0.000,0.268,0.313,0.249,0.365,0.074,0.059,0.333,0.486,0.009
min,10000826.000,2109-03-19 11:40:00,0.000,0.000,70.000,8.000,17.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,12495580.500,2135-12-22 12:53:30,62.500,147.863,120.722,68.969,53.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,15002051.500,2155-11-02 22:04:30,64.019,171.720,129.147,72.429,67.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,17544474.250,2175-11-29 19:46:00,67.750,193.633,134.250,76.590,79.000,1.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
max,19999987.000,2214-12-24 05:14:00,96.000,1213.180,236.000,135.000,106.000,51.000,40.000,27.000,...,1.000,43.000,24.000,21.000,30.000,8.000,10.000,26.000,32.000,7.000
std,2893254.342,NaN,3.705,44.533,12.304,7.450,19.028,2.791,1.414,0.437,...,0.014,0.809,0.978,0.761,1.069,0.333,0.308,0.959,1.249,0.131


In [21]:
df_omr_pt_ad_diag[df_omr_pt_ad_diag.subject_id  == 10002114]

,subject_id,charttime,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,gender,anchor_age,insurance,language,...,SYM009,SYM010,SYM011,SYM012,SYM013,SYM014,SYM015,SYM016,SYM017,SYM018
12,10002114,2162-02-17 21:53:00,69.097,204.136,128.140,76.176,M,56,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
diag[diag.subject_id  == 10002114].dischtime.min()

'2162-03-04 15:16:00'

In [18]:
s3_key = 'MIMIC/Cleaned/df_omr_pt_ad_diag.csv'

# Save the DataFrame to a CSV in memory (using StringIO)
csv_buffer = StringIO()
df_omr_pt_ad_diag.to_csv(csv_buffer, index=False)

# Upload the CSV data to S3
s3.put_object(Bucket=bucket_name, Key=s3_key, Body=csv_buffer.getvalue())

print(f"DataFrame successfully exported to s3://{bucket_name}/{s3_key}")

DataFrame successfully exported to s3://ml-stack-759578648427-us-west-2/MIMIC/Cleaned/df_omr_pt_ad_diag.csv
